In [14]:
# python -m venv venv
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [15]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [16]:
SCOPES = [
    "https://www.googleapis.com/auth/classroom.courseworkmaterials.readonly",
    "https://www.googleapis.com/auth/classroom.student-submissions.me.readonly",
    "https://www.googleapis.com/auth/classroom.rosters",
    "https://www.googleapis.com/auth/classroom.guardianlinks.me.readonly",
    "https://www.googleapis.com/auth/classroom.guardianlinks.students.readonly",
    "https://www.googleapis.com/auth/classroom.courses",
    "https://www.googleapis.com/auth/classroom.guardianlinks.students",
    "https://www.googleapis.com/auth/classroom.topics.readonly",
    "https://www.googleapis.com/auth/classroom.coursework.me",
    "https://www.googleapis.com/auth/classroom.profile.photos",
    "https://www.googleapis.com/auth/classroom.announcements",
    "https://www.googleapis.com/auth/classroom.courses.readonly",
    "https://www.googleapis.com/auth/classroom.rosters.readonly",
    "https://www.googleapis.com/auth/classroom.profile.emails",
    "https://www.googleapis.com/auth/classroom.courseworkmaterials",
    "https://www.googleapis.com/auth/classroom.coursework.students",
    "https://www.googleapis.com/auth/classroom.student-submissions.students.readonly",
    "https://www.googleapis.com/auth/classroom.topics",
    "https://www.googleapis.com/auth/classroom.push-notifications",
    "https://www.googleapis.com/auth/classroom.announcements.readonly",
]

In [17]:
def get_creds():
    """Shows basic usage of the Classroom API.
    Prints the names of the first 10 courses the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
        #atualiza o token, caso novos escopos tenham sido adicionados
        os.remove("token.json")
        with open("token.json", "w") as token_file:
            token_file.write(creds.to_json())
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    return creds

In [18]:
def read_courses():    

    try:
        creds = get_creds()    
        service = build("classroom", "v1", credentials=creds)

        # Call the Classroom API
        results = service.courses().list(pageSize=10).execute()
        courses = results.get("courses", [])

        if not courses:
            print("No courses found.")
            return
        # Prints the names of the first 10 courses.
        print("Courses:")
        for course in courses:
            print(course["name"])

        return courses

    except HttpError as error:
        print(f"\nAn error occurred: {error}\n")

In [19]:
courses = read_courses()

Courses:
Minicurso 2 - MdA 2024
UECE - Estágio Não Obrigatório - 2023.2
UECE - Estágio Obrig. 2023-2
UECE - OC-2023.2
Tópicos IC - 2024.2
LSI 2023
UECE-RNA/IC2023.1
Estatística para a Pesquisa Educacional I / Introdução à Estatística
TEIA2022-DL
IA2022-KRR


In [20]:
id_course = courses[1]["id"] 
print(id_course)

630681386593


In [21]:
def read_topics(id):    

    try:
        creds = get_creds()
        service = build("classroom", "v1", credentials=creds)

        # Call the Classroom API
        results = service.courses().list(pageSize=10).execute()
        courses = results.get("courses", [])

        course_id = id

        # Lista os tópicos do curso
        topics = service.courses().topics().list(courseId=course_id).execute()
        for topic in topics.get("topic", []):
            print(f'Topic: {topic["name"]}')

        # Lista os trabalhos do curso
        course_work = service.courses().courseWork().list(courseId=course_id).execute()
        for work in course_work.get("courseWork", []):
            print(f'Course Work: {work["title"]}')

    except HttpError as error:
        print(f"\nAn error occurred: {error}\n")

In [22]:
read_topics(0)


An error occurred: <HttpError 404 when requesting https://classroom.googleapis.com/v1/courses/0/topics?alt=json returned "Requested entity was not found.". Details: "Requested entity was not found.">



In [26]:
def read_announcments():
    try:
        creds = get_creds()
        service = build("classroom", "v1", credentials=creds)

        # Fetch the list of courses
        results = service.courses().list(pageSize=10).execute()
        courses = results.get("courses", [])

        if not courses:
            print("No courses found.")
            return

        # Select the first course for demonstration
        course_id = courses[0]["id"]

        # Fetch the announcements of the selected course
        announcements = service.courses().announcements().list(courseId=course_id).execute()
        for announcement in announcements.get("announcements", []):
            print(f'Announcement: {announcement["text"]}')

    except HttpError as error:
        print(f"\nAn error occurred: {error}\n")

In [27]:
def read_rosters():
    try:
        creds = get_creds()
        service = build("classroom", "v1", credentials=creds)

        # Fetch the list of courses
        results = service.courses().list(pageSize=10).execute()
        courses = results.get("courses", [])

        if not courses:
            print("No courses found.")
            return

        # Select the first course for demonstration
        course_id = courses[0]["id"]

        # Fetch the teachers of the selected course
        print("Teachers:")
        teachers = service.courses().teachers().list(courseId=course_id).execute()
        for teacher in teachers.get("teachers", []):
            full_name = teacher.get('profile', {}).get('name', {}).get('fullName', 'Unavailable')
            email = teacher.get('profile', {}).get('emailAddress', 'Unavailable')
            print(f"Type: Teacher, Name: {full_name}, Email: {email}")

        # Fetch the students of the selected course
        print("Students:")
        students = service.courses().students().list(courseId=course_id).execute()
        for student in students.get("students", []):
            print(f"Type: Student, Name: {student['profile']['name']['fullName']}, Email: {student['profile']['emailAddress']}")

    except HttpError as error:
        print(f"\nAn error occurred: {error}\n")

In [28]:
read_topics()
read_announcments()
read_rosters()

TypeError: read_topics() missing 1 required positional argument: 'id'